1. Analyzing data

In [3]:
import pandas as pd

movies = pd.read_csv("movies2/movies.csv")


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [6]:
movies["movieId"].unique().shape

(9742,)

In [7]:
ratings = pd.read_csv("movies2/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [9]:
ratings = ratings.drop(columns=["timestamp"],axis=1)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
import numpy as np

ratings_arr = ratings.to_numpy()
np.set_printoptions(suppress=True)

In [11]:
ratings["userId"].nunique()

610

2. Creating User rating matrix which users are in columns and movies in rows and corrosponding binary matrix which marks a user has given a rating or not

In [12]:
import numpy as np

def ratings_matrix(ratings, movies):
    
    user_unique_list, user_indices = np.unique(ratings["userId"], return_inverse=True)
    movie_unique_list, movie_indices = np.unique(movies["movieId"], return_inverse=True)
    
    ratings_mat = np.full((len(movie_unique_list), len(user_unique_list)), np.nan)
    present_mat = np.full((len(movie_unique_list), len(user_unique_list)), 1)

    for idx, row in ratings.iterrows():
        movie_id_idx = np.where(movie_unique_list == row["movieId"])[0]
        user_id_idx = np.where(user_unique_list == row["userId"])[0]  
        ratings_mat[movie_id_idx, user_id_idx] = row["rating"]

    nan_ind = np.argwhere(np.isnan(ratings_mat))
    present_mat[nan_ind[:, 0], nan_ind[:, 1]] = 0
    
    return ratings_mat, present_mat




In [13]:
Y, R = ratings_matrix(ratings, movies)

In [14]:
Y

array([[4. , nan, nan, ..., 2.5, 3. , 5. ],
       [nan, nan, nan, ..., 2. , nan, nan],
       [4. , nan, nan, ..., 2. , nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [15]:
movie_avr_mean =np.nanmean(Y, axis =1)

C:\Users\ACER\AppData\Local\Temp\ipykernel_11212\573461765.py:1: RuntimeWarning: Mean of empty slice
  movie_avr_mean =np.nanmean(Y, axis =1)


In [16]:
movie_avr_mean=movie_avr_mean.reshape(-1,1)
movie_avr_mean

array([[3.92093023],
       [3.43181818],
       [3.25961538],
       ...,
       [3.5       ],
       [3.5       ],
       [4.        ]])

In [17]:
Y.shape

(9742, 610)

In [18]:
Y

array([[4. , nan, nan, ..., 2.5, 3. , 5. ],
       [nan, nan, nan, ..., 2. , nan, nan],
       [4. , nan, nan, ..., 2. , nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [19]:
Ynorm = Y - movie_avr_mean

In [20]:
Ynorm

array([[ 0.07906977,         nan,         nan, ..., -1.42093023,
        -0.92093023,  1.07906977],
       [        nan,         nan,         nan, ..., -1.43181818,
                nan,         nan],
       [ 0.74038462,         nan,         nan, ..., -1.25961538,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

In [21]:
movies["ratings"] = movie_avr_mean


In [22]:
movies.head()

,movieId,title,genres,ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429


In [23]:
movies.describe()

,movieId,ratings
count,9742.000000,9724.000000
mean,42200.353623,3.262448
std,52160.494854,0.869874
min,1.000000,0.500000
25%,3248.250000,2.800000
50%,7300.000000,3.416667
75%,76232.000000,3.911765
max,193609.000000,5.000000


3. Cost finction for compare predicted values and actual ones

In [24]:
def cost(W, X, Y , b, R, lambda_):    
     
    fwb = tf.linalg.matmul(X, tf.transpose(W)) + b
    z = (fwb - Y)*R
    J = 0.5 * tf.reduce_sum(z**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    
    return J

4. creating function for learning X, W, b weights by gradient descent

In [25]:
import tensorflow as tf
import keras

def find_WXb(Ynorm,R, num_features, iterations, lambda_):
    num_movies, num_users = Y.shape
    
    tf.random.set_seed(1234)
    W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
    X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
    b = tf.Variable(tf.random.normal((1,num_users),   dtype=tf.float64),  name='b')
    
    optimizer = keras.optimizers.Adam(learning_rate=1e-1)

    Ynorm_without_nan = np.nan_to_num(Ynorm, nan=0)
    
    for iter in range(iterations):
        
        with tf.GradientTape() as tape:    
            cost_value = cost(W, X, Ynorm_without_nan, b, R, lambda_)
    
        grads = tape.gradient( cost_value, [X,W,b] )
        optimizer.apply_gradients( zip(grads, [X,W,b]) )
    
        if iter % 20 == 0:
            print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
 
    return W, X, b

In [26]:
W1, X1, b1 = find_WXb(Ynorm,R, 10, 200, 1)

Training loss at iteration 0: 635638.8
Training loss at iteration 20: 41927.6
Training loss at iteration 40: 27742.7
Training loss at iteration 60: 22069.0
Training loss at iteration 80: 19312.1
Training loss at iteration 100: 17896.2
Training loss at iteration 120: 17141.4
Training loss at iteration 140: 16687.4
Training loss at iteration 160: 16379.3
Training loss at iteration 180: 16153.8


In [27]:
W2, X2, b2 = find_WXb(Ynorm,R, 100, 200, 1)

Training loss at iteration 0: 5660823.1
Training loss at iteration 20: 285181.5
Training loss at iteration 40: 110018.3
Training loss at iteration 60: 54110.6
Training loss at iteration 80: 31053.4
Training loss at iteration 100: 19894.7
Training loss at iteration 120: 13879.4
Training loss at iteration 140: 10407.9
Training loss at iteration 160: 8311.6
Training loss at iteration 180: 7001.0


In [28]:
p1 = np.matmul(X1.numpy(), np.transpose(W1.numpy())) + b1.numpy()

pm1 = p1 + movie_avr_mean
pm1

array([[4.81680129, 4.35638059, 1.4205668 , ..., 2.57051367, 3.64609966,
        4.80133663],
       [4.09485427, 4.09829929, 1.84583882, ..., 2.27256777, 3.36588013,
        3.57835491],
       [4.33146476, 3.87430952, 1.15723644, ..., 2.58058484, 3.41346215,
        3.80389075],
       ...,
       [4.32775867, 3.78202706, 1.98029939, ..., 3.4176055 , 3.3355664 ,
        3.62181085],
       [4.32797447, 3.78212874, 1.9803777 , ..., 3.41750422, 3.33559912,
        3.62191299],
       [4.8586558 , 4.20620899, 2.37356582, ..., 3.93511268, 3.80847127,
        4.22480559]])

In [29]:
p2 = np.matmul(X2.numpy(), np.transpose(W2.numpy())) + b2.numpy()

pm2 = p2 + movie_avr_mean
pm2

array([[4.0658699 , 3.65639523, 3.94000628, ..., 2.40473547, 3.09225092,
        5.09150447],
       [4.15994658, 4.16480117, 2.06204179, ..., 2.00005927, 3.90128804,
        2.992444  ],
       [3.99320373, 2.95720383, 1.98456541, ..., 2.04820208, 2.99805979,
        3.35433073],
       ...,
       [4.39439229, 3.56588274, 2.4487714 , ..., 3.39043286, 3.38973501,
        3.65533532],
       [4.39413164, 3.56598366, 2.44895957, ..., 3.39046451, 3.3898049 ,
        3.65531221],
       [4.88722974, 4.06103006, 2.95711439, ..., 3.90951303, 3.88220221,
        4.14677946]])

In [30]:
Y

array([[4. , nan, nan, ..., 2.5, 3. , 5. ],
       [nan, nan, nan, ..., 2. , nan, nan],
       [4. , nan, nan, ..., 2. , nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

5. Here Onwards, creating functions for finding similar movies by movie features and recommend movies for a new user from his curunt interest.

In [31]:
def find_similar_movie_idx(X, movie_index):
    movie = X[movie_index]
    movies_num = X.shape[0]
    simi_list = []
    for i in range(movies_num):
        similarity = np.linalg.norm(movie - X[i])**2
        simi_list.append(similarity)
    simi_list = np.array(simi_list)
    simi_movie_indexes = np.argsort(simi_list)[1:11]

    return simi_movie_indexes

In [32]:
import re

def clean_titles(title):
    
    cleaned_title = re.sub("[^a-zA-Z0-9 ]","",title)
    return cleaned_title

In [33]:
movies_cleaned = movies
cleaned_titles = movies_cleaned["title"].apply(clean_titles)
movies_cleaned.loc[:, "cleaned_title"] = cleaned_titles
movies_cleaned.head()

,movieId,title,genres,ratings,cleaned_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,3.071429,Father of the Bride Part II 1995


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorized_titles = vectorizer.fit_transform(movies_cleaned["cleaned_title"])


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

def search_movie(title):
    cleaned_title = clean_titles(title)
    vectorized_title = vectorizer.transform([cleaned_title])
    similarity = cosine_similarity(vectorized_title, vectorized_titles).flatten()
    indices = np.argsort(similarity)[::-1][:5]
    return movies_cleaned.iloc[indices]

In [36]:
def similar_movies(title):
    movie_id = search_movie(title)["movieId"].iloc[0]
    movie_unique_list, movie_indices = np.unique(movies["movieId"], return_inverse=True)
    
    movie_idx = np.where(movie_unique_list == movie_id)[0][0]
    similar_movies_idx = find_similar_movie_idx(X2, movie_idx)

    
    return movies.iloc[similar_movies_idx]

6. Function to find new users W,b weights using learned movie features

In [37]:
def find_Wb(Ynorm,R, num_features, iterations, lambda_,X):
    num_movies, num_users = Ynorm.shape
    
    tf.random.set_seed(1234) # for consistent results
    W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
    b = tf.Variable(tf.random.normal((1,num_users),   dtype=tf.float64),  name='b')
    

    optimizer = keras.optimizers.Adam(learning_rate=1e-1)

    Ynorm_without_nan = np.nan_to_num(Ynorm, nan=0)
    
    for iter in range(iterations):
        with tf.GradientTape() as tape:
            cost_value = cost(W, X, Ynorm_without_nan, b, R, lambda_)

        grads = tape.gradient( cost_value, [W,b] )    
        optimizer.apply_gradients( zip(grads, [W,b]) )
    
        if iter % 20 == 0:
            print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
 
    return W, b

7. Creating a class for handle user functions

In [38]:
class User:
    
    def __init__(self):
        self.movies = movies
        self.movie_avr_mean = movie_avr_mean
        self.X2 = X2
        
        self.user_arr = np.full((1, movies.shape[0]), np.nan)
        self.user_present = np.full((1, movies.shape[0]), 1)
        
        self.movie_unique_list, self.movie_indices = np.unique(movies["movieId"], return_inverse=True)
        self.preds = None
        self.R = None
    
    def give_ratings(self, movie_id, ratings):
        movie_idx = np.where(self.movie_unique_list == movie_id)[0][0]
        if 0 <= ratings <= 5:
            self.user_arr[0][movie_idx] = ratings
        else:
            print("Rating is beyond limits")
    
    def get_predictions(self):
        nan_ind = np.argwhere(np.isnan(self.user_arr))
        self.user_present[0,nan_ind[:, 1]] = 0
        
        Y = self.user_arr.reshape(-1, 1)
        R = self.user_present.reshape(-1, 1)
        self.R = R
        Ynorm = Y - self.movie_avr_mean
        
        W_user, b_user = find_Wb(Ynorm, R, 100, 200, 1, self.X2)
        prediction = np.matmul(self.X2.numpy(), np.transpose(W_user.numpy())) + b_user.numpy()
        self.preds = prediction + self.movie_avr_mean
        return self.preds
    
    def get_recommends(self):
        if self.preds is not None:
            #reco = self.preds * np.logical_not(self.R)
            reco_indices = np.argsort(self.preds.flatten())[-10:][::-1]
            dataframe = self.movies[["title", "genres"]].copy()
            return dataframe.iloc[reco_indices]
        else:
            print("Calculate predictions first")

In [39]:
user1 = User()
user1.give_ratings(1,4.5)
user1.give_ratings(2,4.0)
user1.give_ratings(3,3)
user1.give_ratings(4,2)



In [40]:
user1.get_predictions()

Training loss at iteration 0: 3557.8
Training loss at iteration 20: 3503.1
Training loss at iteration 40: 3500.4
Training loss at iteration 60: 3500.1
Training loss at iteration 80: 3500.1
Training loss at iteration 100: 3500.1
Training loss at iteration 120: 3500.1
Training loss at iteration 140: 3500.1
Training loss at iteration 160: 3500.1
Training loss at iteration 180: 3500.1


array([[4.42434261],
       [3.89797975],
       [3.01940319],
       ...,
       [3.33560505],
       [3.33559781],
       [3.83660852]])

In [41]:
user1.get_recommends()

,title,genres
816,"Innocents, The (1961)",Drama|Horror|Thriller
5452,Mutiny on the Bounty (1962),Adventure|Drama|Romance
2211,Niagara (1953),Drama|Thriller
2499,For All Mankind (1989),Documentary
2587,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
5020,"Parallax View, The (1974)",Thriller
7565,Chalet Girl (2011),Comedy|Romance
5293,This Gun for Hire (1942),Crime|Film-Noir|Thriller
4037,"Chosen, The (1981)",Drama
4598,Scrooge (1970),Drama|Fantasy|Musical


8. User Interface for find similar movies

In [42]:
import ipywidgets as widgets
from IPython.display import display


input_title = widgets.Text(value="",
                           description="Title",
                           disabled=False)
sim_movies = widgets.Output()


def on_button_click(b):
    with sim_movies:
        sim_movies.clear_output()
        display(similar_movies(input_title.value))

search_button = widgets.Button(description="Search")

search_button.on_click(on_button_click)

display(input_title, search_button, sim_movies)


Text(value='', description='Title')

Button(description='Search', style=ButtonStyle())

Output()

9. User interface to recommend movies learning from his current interests

In [43]:
user1 = User()

input_title2 = widgets.Text(value="",
                           description="Title",
                           disabled=False)
input_rating = widgets.Text(value="",
                           description="Rating",
                           disabled=False)
movies_ = widgets.Output()
rec_movies = widgets.Output()
confirm = widgets.Output()

def on_search_button_click(b):
    with movies_:
        global movie
        movies_.clear_output()
        movie = search_movie(input_title2.value).iloc[0]
        display(movie)

def on_ok_button_click(b):
    with confirm:
        confirm.clear_output()
        user1.give_ratings(movie["movieId"], float(input_rating.value))
        display("recorded :" ,input_rating.value)
    
def on_recommended_movie_button_click(b):
    with rec_movies:
        rec_movies.clear_output()
        user1.get_predictions()
        display(user1.get_recommends())
        
search_button = widgets.Button(description="Search")
ok_button = widgets.Button(description="OK")
recommended_movie_button = widgets.Button(description="Get Recommend")

search_button.on_click(on_search_button_click)
ok_button.on_click(on_ok_button_click)
recommended_movie_button.on_click(on_recommended_movie_button_click)


display(input_title2, search_button, movies_)
display( input_rating, ok_button, confirm)
display(recommended_movie_button,rec_movies)

Text(value='', description='Title')

Button(description='Search', style=ButtonStyle())

Output()

Text(value='', description='Rating')

Button(description='OK', style=ButtonStyle())

Output()

Button(description='Get Recommend', style=ButtonStyle())

Output()